# Integrating GRNs with multi-omics data
Author: Romana T. Pop^1^

1. Centre for Molecular Medicine Norway (NCMM), Faculty of Medicine, University of Oslo, Oslo, Norway

We use MOFA+ to investigate the contribution of GRNs to JDR models and their association with patient survival. We run MOFA+ separately on all omics data types with no GRN information and compared the results with MOFA+ models that included network metrics (indegree, outdegree, and both).

In [ ]:
# ensure environment is clean
rm(list = ls())

# install MARMOT
# uncomment if not installed
#library(devtools)
#install_github("rtpop/MARMOT", ref = "v0.0.1")

# load libraries
library(MARMOT)
library(tidyverse)
library(reshape2)
library(MOFA2)

In [ ]:
# some intermediate files are provided for ease, set this parameter to FALSE
# if you do not wish to use them and wish to compute them again instead
precomputed <- TRUE

# setting working directory
wd <- "../JDRnet"
setwd(wd)

# specify directory for results to be saved
tcga_res <- "TCGA"
gep_res <- "GEPliver"

# defining vector of cancer names for which to do the analysis
cancers_tcga <- c("aml", "breast", "colon", "gbm", "kidney", "liver", "lung",
          "melanoma", "ovarian", "sarcoma")

# defining names for the JDR models that we will run
model <- c("nonet", "indeg", "out", "both")

# define vector of omic names that will be used
omics_tcga <- c("expression", "methylation", "miRNA", "indegree", "outdegree")

In [ ]:
# running MOFA models
for (cancer in cancers_tcga) {
  load(file.path(tcga_res, paste0(cancer, "_omics_pca.Rda")))

  #nonet
  data_nonet <- omics[1:3]
  mofa_nonet <- run_mofa2(data_nonet, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_nonet, file = file.path(tcga_res, paste0("MOFA_", cancer, "_pca_nonet.Rda")))

  # with indeg
  data_indeg <- omics[-5]
  mofa_indeg <- run_mofa2(data_indeg, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_indeg, file = file.path(tcga_res, paste0("MOFA_", cancer, "_pca_indeg.Rda")))

  # with outdeg
  data_out <- omics[-4]
  mofa_out <- run_mofa2(data_out, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_out, file = file.path(tcga_res, paste0("MOFA_", cancer, "_pca_out.Rda")))

  # with both
  data_both <- omics
  mofa_both <- run_mofa2(data_both, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_both, file = file.path(tcga_res, paste0("MOFA_", cancer, "_tcga_pca_both.Rda")))
}

We now perform univariate cox regression for each factor to determine their association with patient survival. We compare the association of the factors to patient survival in the models without GRNs and the models with GRNs, reproducing Figure 3 from the paper.

In [ ]:
# Define model types
model_types <- c("nonet", "indeg", "out", "both")

# Define file paths for MOFA models and survival data
mofa_files <- lapply(model_types, function(model) {
  file.path(tcga_res, paste0("MOFA_", cancers_tcga, "_pca_", model, ".Rda"))
})
mofa_files <- do.call(c, mofa_files)

surv_files <- file.path(tcga_res, paste0(cancers_tcga, "_surv.Rda"))

# Initialize the survival data frame
surv_df <- data.frame()

# process each cancer for each model type
new_surv_df <- Map(function(cancer, mofa_files, surv_file) {
  message("Processing: ", cancer)

  # Load MOFA models and survival data
  models <- lapply(mofa_files, load)
  models <- lapply(models, get)
  load(surv_file) # loads into 'surv' variable

  # Getting factors
  factors <- lapply(models, function(model) get_factors(model)[[1]])

  # Perform survival associations
  cox_models <- lapply(factors, surv_association, surv = surv, univariate = TRUE)

  # Compare survival models
  df <- surv_compare(models = cox_models, model_labels = model_types,
                     univariate = TRUE, method = "BH")
  df$cancer <- cancer

  # Save individual cox models
  cox_all <- setNames(cox_models, model_types)
  save(cox_all, file = file.path(tcga_res, paste0(cancer, "_cox_models_PCA.Rda")))

  return(df)
}, rep(cancers_tcga, each = length(model_types)), seq(1, length(mofa_files), length(cancers_tcga)), surv_files)

# Combine all results into a single data frame
surv_df <- do.call(rbind, new_surv_df)

# Save the combined survival data frame
save(surv_df, file = file.path(tcga_res, "MOFA_surv_df_all_PCA.Rda"))

In [ ]:
# plotting
if(!exists("surv_df")){
  load(file.path(tcga_res, "MOFA_surv_df_all_PCA.Rda"))
}

#get models to compare
model_comp <- setdiff(model, "nonet")

#set colours
cols <- palette("Dark2")

for(i in model_comp){
  models_to_compare <- c("nonet", i)
  p <- surv_compare_dotplot(surv_df = surv_df, models_to_compare = models_to_compare, 
                            colours = c(cols[8], "grey", cols[1]))

  ggsave(p, file = paste0(figure_dir, paste0("surv_compare_",models_to_compare[2],"_tcga.pdf")))
}